In [ ]:
import altair as alt
import polars as pl
from plot_data import month_order
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder

from iup.utils import date_to_season

In [ ]:
SEASON_START_MONTH = 7  # July
END_MONTH = 9  # 9 months after July, i.e., April
MIN_T = 1 - END_MONTH
months = month_order(season_start_month=SEASON_START_MONTH)

data = (
    pl.read_parquet("../data/raw.parquet")
    .filter(pl.col("geography_type") == pl.lit("admin1"))
    .with_columns(
        season=date_to_season(
            pl.col("time_end"), season_start_month=SEASON_START_MONTH
        ),
        t=pl.col("time_end")
        .dt.to_string("%b")
        .map_elements(lambda x: months.index(x) - END_MONTH, pl.Int64),
    )
    .filter(pl.col("t").is_between(MIN_T, 0))
    # remove partial seasons
    .filter(pl.col("season").is_in(["2008/2009", "2022/2023"]).not_())
    .select(["season", "geography", "t", "estimate"])
    # go to long format
    .with_columns(pl.format("t={}", pl.col("t")))
    .pivot(on="t", values="estimate")
    # this is a kludge: really should impute these values
    .drop_nulls()
    .sort(["season", "geography"])
)

data

season,geography,-8,-7,-6,-5,-4,-3,-2,-1,0
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2009/2010""","""Alabama""",0.007,0.082,0.255,0.334,0.354,0.39,0.405,0.41,0.413
"""2009/2010""","""Alaska""",0.007,0.083,0.241,0.319,0.349,0.318,0.329,0.332,0.337
"""2009/2010""","""Arizona""",0.007,0.087,0.249,0.336,0.364,0.353,0.371,0.386,0.394
"""2009/2010""","""California""",0.007,0.059,0.185,0.264,0.303,0.325,0.338,0.344,0.349
"""2009/2010""","""Colorado""",0.01,0.085,0.283,0.376,0.4,0.384,0.395,0.406,0.413
…,…,…,…,…,…,…,…,…,…,…
"""2021/2022""","""Vermont""",0.014,0.086,0.339,0.482,0.533,0.569,0.58,0.585,0.589
"""2021/2022""","""Virginia""",0.016,0.108,0.302,0.403,0.445,0.475,0.496,0.507,0.513
"""2021/2022""","""Washington""",0.016,0.104,0.3,0.4,0.445,0.482,0.503,0.515,0.521


In [121]:
data.describe()

statistic,season,geography,-8,-7,-6,-5,-4,-3,-2,-1,0
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""599""","""599""",599.0,599.0,599.0,599.0,599.0,599.0,599.0,599.0,599.0
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,0.012479,0.08002,0.260876,0.352551,0.384643,0.413038,0.429144,0.438209,0.44413
"""std""",null,null,0.005434,0.019627,0.043546,0.054967,0.058102,0.059877,0.061245,0.062606,0.06362
"""min""","""2009/2010""","""Alabama""",0.004,0.036,0.079,0.127,0.147,0.18,0.18,0.18,0.18
"""25%""",null,null,0.009,0.067,0.233,0.317,0.346,0.373,0.387,0.395,0.399
"""50%""",null,null,0.012,0.077,0.259,0.352,0.384,0.411,0.428,0.438,0.443
"""75%""",null,null,0.015,0.09,0.288,0.39,0.425,0.454,0.469,0.479,0.484
"""max""","""2021/2022""","""Wyoming""",0.04,0.166,0.417,0.508,0.545,0.582,0.614,0.631,0.642


In [122]:
rf = RandomForestRegressor(oob_score=True)
enc = OrdinalEncoder()

features = ["season", "geography"] + [str(x) for x in range(MIN_T, 0)]
X = data.select(features).to_numpy()
y = data.select("0").to_series().to_numpy()

enc.fit(X)
rf.fit(enc.transform(X), y)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [123]:
print(*zip(features, rf.feature_importances_))

('season', np.float64(0.0009951534069092209)) ('geography', np.float64(0.000750705007692485)) ('-8', np.float64(0.0008000247693484428)) ('-7', np.float64(0.0010690406672139152)) ('-6', np.float64(0.0019865142381866223)) ('-5', np.float64(0.0008931990335107375)) ('-4', np.float64(0.0022530392683649055)) ('-3', np.float64(0.008947180353817901)) ('-2', np.float64(0.011269862330927184)) ('-1', np.float64(0.9710352809240286))


In [124]:
# post hoc (but OOB) comparison of end-of-season predictions
alt.Chart(data.with_columns(pred=rf.oob_prediction_)).mark_point().encode(
    alt.X("0"), alt.Y("pred")
)

alt.Chart(...)

In [125]:
# distribution of end-of-season errors
alt.Chart(
    data.with_columns(pred=rf.oob_prediction_).with_columns(
        error=pl.col("pred") - pl.col("0")
    )
).mark_bar().encode(alt.X("error", bin=True), alt.Y("count()"))

alt.Chart(...)

In [156]:
def forecast(
    forecast_t: int, target_t: int = 0, target_season: str = "2021/2022", data=data
):
    assert forecast_t >= MIN_T

    features = ["season", "geography"] + [str(x) for x in range(MIN_T, forecast_t + 1)]

    data_fit = data.filter(pl.col("season") != pl.lit(target_season))
    data_pred = data.filter(pl.col("season") == pl.lit(target_season))

    enc = OrdinalEncoder()
    enc.fit(data.select(features).to_numpy())

    X_fit = data_fit.select(features).to_numpy()
    y_fit = data_fit.select("0").to_series().to_numpy()

    rf = RandomForestRegressor()
    rf.fit(enc.transform(X_fit), y_fit)

    # make the forecast
    X_pred = data_pred.select(features).to_numpy()
    y_pred = rf.predict(enc.transform(X_pred))

    importances = pl.DataFrame(
        {
            "forecast_t": forecast_t,
            "feature": features,
            "importance": rf.feature_importances_,
        }
    )

    preds = data_pred.select(["season", "geography"]).with_columns(
        forecast_t=forecast_t, pred=y_pred
    )

    return preds, importances


results = [forecast(x) for x in range(MIN_T, 0 + 1)]
forecasts = pl.concat([x[0] for x in results])
importances = pl.concat([x[1] for x in results])

In [157]:
(
    alt.Chart(forecasts)
    .mark_line(point=True)
    .encode(alt.X("forecast_t"), alt.Y("pred"), alt.Facet("geography"))
)

alt.Chart(...)

In [154]:
errors = (
    forecasts.filter(pl.col("forecast_t") != 0)
    .join(
        forecasts.filter(pl.col("forecast_t") == 0)
        .drop("forecast_t")
        .rename({"pred": "true"}),
        on=["season", "geography"],
    )
    .with_columns(error=pl.col("pred") - pl.col("true"))
)

alt.Chart(errors).mark_bar().encode(
    alt.X("error", bin=alt.Bin(step=0.01)), alt.Y("count()"), alt.Facet("forecast_t")
)

alt.Chart(...)

In [163]:
alt.Chart(
    importances.group_by(
        pl.col("forecast_t"),
        pl.col("feature").replace({str(x): "coverage" for x in range(MIN_T, 1)}),
    ).agg(pl.col("importance").sum())
).mark_line().encode(alt.X("forecast_t"), alt.Y("importance"), alt.Color("feature"))

alt.Chart(...)